## Welcome!

### Some Google Colab hacks
* To run a cell, hit `Ctrl + Shift + Enter`
* To run the whole sheet, hit `Ctrl + F9`
* To run the cells above, hit `Ctrl + F8`
* To run the cells below, hit `Ctrl + F10`


# Install and import `pygismo` and other packages

`pygismo` is a small package, unknown to Google. Therefore, we have to install it locally inside the Notebook

In [2]:
# !pip install pygismo

# We use the development version of pygismo
from google.colab import drive
drive.mount('/content/drive/')

!pip install "/content/drive/MyDrive/Google Colab/pygismo/wheels/pygismo-23.9.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl"

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
Processing ./drive/MyDrive/Google Colab/pygismo/wheels/pygismo-23.9.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
pygismo is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.


Now we can import it as a package

In [3]:
import pygismo as gs

We also import `numpy` and `matplotlib`

In [4]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d

# Constructing a 2D tensor B-spline basis

A 2D B-spline basis is a *tensor-product* basis. This means that it has two knot vectors $\Xi_u$ and $\Xi_v$. In this case, we choose:

$\Xi_u = \{0,0,0,0.25,0.50,0.75,1,1,1\}$

$\Xi_v = \{0,0,0,0,0.25,0.50,0.75,1,1,1,1\}$


In [5]:
kv_u = gs.nurbs.gsKnotVector(np.array([0,0,0,0.25,0.50,0.75,1,1,1]),2)
kv_v = gs.nurbs.gsKnotVector(np.array([0,0,0,0,0.25,0.50,0.75,1,1,1,1]),3)

Then, we define the tensor B-spline basis

In [6]:
tbasis = gs.nurbs.gsTensorBSplineBasis2(kv_u,kv_v)
print('The basis has size: ', tbasis.size())

The basis has size:  42


# Constructing a 2D Truncated Hierarchical B-spline basis

To construct a Truncated Hierarchical B-spline basis, we simply intialize the `gsTHBSplineBasis2` using the tensor B-spline basis constructed before:

In [20]:
thbbasis = gs.hsplines.gsTHBSplineBasis2(tbasis)
hbbasis = gs.hsplines.gsHBSplineBasis2(tbasis)

This THB spline basis just has one level. If we want to perform a refinement, hence introduce functions from another level, we call `pygismo.hsplines.gsTHBSplineBasis2.refineElements`. This functions takes a list as argument, specified as follows:

`box = [level,xlow,ylow,xupp,yupp]`

Here, `level` is the level of the element to be inserted, (`xlow`,`ylow`) are the element indices of the lower corner and (`xupp`,`yupp`) are the element indices of the upper corner. These indices are ordered according to the knot vectors. The first unique knot has index 0, the second has index 1, and so forth.

For example to refine the element in the upper-left and upper-left corners of our domain, we activate an element in level 1 with the indices corresponding to $2\times$the element index in level 0:

In [15]:
thbbasis.refineElements([1,0,6,2,8])
thbbasis.refineElements([1,6,0,8,2])

thbbasis.refineElements([2,0,0,4,4])
thbbasis.refineElements([2,14,14,16,16])

print('The basis has size: ', thbbasis.size())

The basis has size:  69


# Plotting the basis in Paraview

We can plot the basis using Paraview, as done in the 2D basis example:

In [18]:
# gs.io.gsWriteParaview(tbasis,"/content/drive/MyDrive/Google Colab/pygismo_demo/output/tbasis")
gs.io.gsWriteParaview(thbbasis,"/content/drive/MyDrive/Google Colab/pygismo_demo/thb/thbbasis")